In [1]:
import pandas as pd
import numpy as np
import copy 

In [2]:
df = pd.read_csv("not_accessed_not_expiring_2022_10_14_w_r.csv")

In [3]:
datasets = df["dataset"]
location = df["RSE Name"]
size_on_tape = df["dataset_size_in_rse_tb"]
tier = [ i[-1] for i in df["dataset"].str.split("/").tolist() ]

In [4]:
exceptions = pd.read_csv("exceptions.txt")

In [5]:
exception_list = exceptions["item"]

## Total size on tape

In [6]:
np.sum(size_on_tape)

19915.4273

In [7]:
keep = np.full( np.size(datasets), False )

## Breakdown of campaign to "human readable" format

In [8]:
categories = {

    "Skims":["USER"],
    "Commissioning + Cosmics":[
       "Commissioning08",     
       "Commissioning10",  
       "Commissioning11",  
       "Commissioning12",  
       "Commissioning2015",
       "Commissioning2017",
       "Commissioning2018",
       "Commissioning2021",
       "Commissioning2022",
    ],
    
    "Cosmics MC":[
        "RunIISpring18CosmicDR"
    ],
        
    "Run 3":[
        "Run3Summer19DRPremix",       
        "Run3Summer19DR",  
        "Run3Winter20",
        "Run3Winter20DRPremixMiniAOD",
        "Run3Winter21DRMiniAOD",      
        "Run3Summer21",               
        "Run3Winter21",               
        "Run3Summer21GS",             
        "Run3Summer21PrePremix",      
        "Run3Summer21wmLHEGS",        
        "Run3Summer22",                  
    ],
    
    "Run 2 legacy MC":[
        "Summer19UL",
        "Summer20UL",
        "Spring21UL",
    ],
    
    "Run 2 legacy data":[
        "UL2016",
        "UL2017",
        "UL2018",
    ],
    
    "Run 2 pre-legacy data":[
        "23Sep2016",
        "18Apr2017",
        "07Aug17",
        "12Sep2017",
        "17Nov2017",
        "09May2018",
        "06Jun2018",
        "17Sep2018",
        "22Jan2019",
        "Run2018D-PromptReco",
        "RunIIWinter17DR",
    ],

    "Run 2 pre-legacy MC":[
        "RunIISpring15PrePremix",    
        "RunIIFall15DR76",           
        "RunIISummer16DR80",         
        "RunIISummer16DR80Premix",   
        "RunIIFall17GS",             
        "RunIIFall17DRPremix",       
        "RunIISummer17PrePremix",    
        "RunIIFall17FSPremix",       
        "RunIIFall17FSPrePremix",    
        "RunIIFall18wmLHEGS",        
        "RunIIFall18GS",             
        "RunIIAutumn18DRPremix",     
        "RunIIAutumn18DR",           
        "RunIIAutumn18RECOBParking", 
        "RunIIAutumn18FSPremix",        
        "RunIIWinter19PFCalib17wmLHEGS",
        "RunIISpring18DRPremix",        
        "RunIIFall17DRStdmix",          
        "RunIISummer17DRStdmix",        
        "RunIISpring18DR",
        "RunIISpring18GS",
        "RunIISpring15DR74",            
        "RunIISummer16FSPremix",        
        "RunIISummer15wmLHEGS",         
        "RunIIWinter19CosmicDR",        
        "RunIIFall17wmLHEGS",           
        "RunIIWinter19PFCalib16wmLHEGS",
        "RunIIFall17wmLHEGENOnly",      
        "RunIIWinter15GS",
        "RunIISpring16FSPremix",
        "RunIIWinter19PFCalib16GS",
        "RunIILowPUAutumn18GS",
        "RunIIWinter19PFCalib",
        "RunIISummer15"
    ],
    
    "B-Parking":[
        "ParkingBPH",
        "SkimBPark"
    ],    
    
    "HIN":[
        "HIRun2010", 
        "HIRun2011", 
        "HIRun2013", 
        "HIRun2015", 
        "PARun2016", 
        "HIRun2018", 
        "HINPbPbAutumn18DR",
        "HiWinter13", 
        "pPb816Summer16DR",
        "HINPbPbWinter16DR",
        "HiFall13DR53X",
        "pAWinter13DR53X",
        "XeXeRun2017",
        "HIRun2013A",
        "HINppWinter16DR",
        "pPb816Spring16GS",
        "HiFall13",
    ],

    "ALCA":[
        "StreamExpressAlignment",
        "TkAlCosmicsInCollisions",
        "ALCARECO"
    ],
    
    "DPOA":[
        "LowPU2010",
        "Run2010",
        "Run2010",
        "Run2010",
        "Run2011",
        "Run2011",
        "Run2011",
        "Run2012",
        "Run2012",
        "Summer11",
        "Summer11Leg",
        "Fall11",
        "Summer11LegDR",
        "Summer12DR53X",
        "Summer11dr53X",
        "Summer12_DR53X",
        "Summer12",
        "Summer12",
        "Summer13dr53X",
        "Run2015C_25ns",
        "Run2015",
        "Run2015",
    ],
    
    "Upgrades":[
        "Phase2HLTTDRWinter20DIGI", 
        "Phase2HLTTDRSummer20ReRECOMiniAOD",
        "PhaseIITDRSpring17DR", 
        "PhaseIIFall16DR82",
        "PhaseIISummer17wmLHEGENOnly",
        "GEM2019Upg14DR",
        "PhaseISpring17DR", 
        "PhaseIFall16DR", 
        "Phase2HLTTDRWinter20GS",
        "PhaseIIMTDTDRAutumn18DR",
        "PhaseIITDRSpring19DR",
        "PhaseIITDRFall17GS",
        "PhaseIITDRFall17DR",
        "PhaseIISpr18AODMiniAOD",
        "TP2023HGCAL",
        "SHCAL2023Upg14",
        "TP2023SHCALDR",
        "PhaseIISummer17GenOnly",
        "Phase2HLTTDRWinter20RECOMiniAOD",
        "PhaseIIFall17D",
        "Muon2023Upg14DR",
        "RunIISummer15",
        "RunIISpring16reHLT80",
        "Upg2023SHCAL14"
    ],
    


    
    "Prompt":[
        "Prompt",
    ],
    
    "LHE":[
        "LHE",
    ],
    
    "Data challenges":[
        "CONTAINER"
    ],

    "Miscellaneous":[
        "CMSSW_1_3_2",
    ],
}



### List of all campaigns

In [9]:
campaign_list = []

for vals in categories.values():
    campaign_list = campaign_list + vals

### Map of campaigns to the category they belong to

In [10]:
cats_map = {}
for val in campaign_list: 
    for cat in categories:
        if val in categories[cat]:
            cats_map[val] = cat

### Map of datasets to the category they belong to

In [11]:
dataset_categories = []
for dataset in datasets:
    found = False
    for category in categories: 
        for campaign in categories[category]:
            if campaign in dataset and not found: 
                dataset_categories.append(category)
                found = True
    if not found:
        dataset_categories.append("Miscellaneous")

In [12]:
df['category'] = dataset_categories

In [13]:
for category in categories:
    isum = np.sum( df[ df["category"] == category]["dataset_size_in_rse_tb"] )
    print( "%30s : %12.1f" % ( category, isum ) )

                         Skims :        150.4
       Commissioning + Cosmics :        557.9
                    Cosmics MC :          0.0
                         Run 3 :        463.4
               Run 2 legacy MC :       1249.8
             Run 2 legacy data :       2626.4
         Run 2 pre-legacy data :       1857.8
           Run 2 pre-legacy MC :       1616.4
                     B-Parking :        199.8
                           HIN :       1980.9
                          ALCA :        205.2
                          DPOA :       2583.3
                      Upgrades :       1217.9
                        Prompt :        549.1
                           LHE :        188.9
               Data challenges :          0.0
                 Miscellaneous :       4468.4


## Decide which datasets to delete and keep

#### Specific exceptions

In [14]:
retained1 = np.in1d(datasets, exception_list)

In [15]:
keep = np.where( retained1 == True, True, keep )

#### Global exceptions

In [16]:
global_exceptions = {}

for campaign in campaign_list:
    global_exceptions[campaign] = []

In [17]:
campaign_tiers_to_keep = [
        ["USER", "USER"],                         #
        ["Commissioning08", "*"],                 #    
        ["Commissioning10", "*"],                 #
        ["Commissioning11", "*"],                 #
        ["Commissioning12", "RECO"],              #
        ["Commissioning2015", "*"],               # 
        ["Commissioning2017", "AOD"],             #
        ["Commissioning2018", "RAW-RECO"],        #
        ["Commissioning2021", "*"],                # 
        ["Commissioning2022", "*"],                # 
        ["Run3Summer19DRPremix", "*"],       #
        ["Run3Summer19DR", "*"],             #
        ["Run3Winter20", "*"],
        ["Run3Winter20DRPremixMiniAOD", "AODSIM"],#
        ["Run3Winter21DRMiniAOD", "AODSIM"],       # 
        ["Run3Summer21", "*"],                    #
        ["Run3Winter21", "*"],                    #
        ["Run3Summer21GS", "*"],                  #
        ["Run3Summer21PrePremix", "*"],           #
        ["Run3Summer21wmLHEGS", "GEN-SIM"],       #
        ["Run3Summer22", "*"],                    #    
        ["Summer20UL", "*"],                       # 
        ["Spring21UL", "*"],                       # 
        ["UL2016", "*"],                           # 
        ["UL2017", "*"],                           # 
        ["UL2018", "*"],                           #
        #["23Sep2016", "AOD"], 
        #["18Apr2017", "AOD"], 
        #["07Aug17", "AOD"], 
        #["12Sep2017", "AOD"], 
        #["17Nov2017", "AOD"], 
        #["09May2018", "AOD"], 
        #["06Jun2018", "AOD"], 
        #["17Sept2018", "AOD"], 
        #["22Jan2019", "AOD"], 
        #["Run2018D-PromptReco", "AOD"], 
        #["Run2018D-PromptReco", "AOD"],    
        ["RunIISpring15PrePremix", "PREMIX"],     #
        #["RunIIFall15DR76", "AODSIM"],            #
        #["RunIISummer16DR80", "AODSIM"],          #     
        ["RunIISummer16DR80Premix", "PREMIX"],     #
        #["RunIIFall17GS", "*"],                   #
        ["RunIIFall17DRPremix", "PREMIX"],         # 
        ["RunIISummer17PrePremix", "PREMIX"],      #
        ["RunIIFall17FSPremix", "PREMIX"],             #
        ["RunIIFall17FSPrePremix", "PREMIX"],          #
        #["RunIIFall18wmLHEGS", "*"],              #
        #["RunIIFall18GS", "*"],                   #
        ["RunIIAutumn18DRPremix", "PREMIX"],      # 
        #["RunIIAutumn18DR", "AODSIM"],            #
        ["RunIIAutumn18RECOBParking", "AODSIM"],  #
        #["RunIIAutumn18FSPremix", "*"],      #
        ["RunIIWinter19PFCalib17wmLHEGS", "LHE"], #
        ["RunIISpring18DRPremix", "PREMIX"],      #
        #["RunIIFall17DRStdmix", "*"],        #
        #["RunIISummer17DRStdmix", "*"],      #
        #["RunIISpring18DR", "AODSIM"],            #
        #["RunIISpring15DR74", "AODSIM"],          #
        #["RunIISummer16FSPremix", "*"],      #
        ["RunIISummer15wmLHEGS", "LHE"],          #
        #["RunIIWinter19CosmicDR", "*"],           #
        ["RunIIFall17wmLHEGS", "LHE"],            #    
        #["RunIIWinter19PFCalib16wmLHEGS", "*"],   #
        ["RunIIFall17wmLHEGENOnly", "LHE"],       #
        ["RunIIWinter15GS", "GEN-SIM"],           #
        ["SkimBPark", "AOD"],
        ["ParkingBPH", "AOD"],                     # 
        ["HIRun2010", "RECO"],                    # 
        ["HIRun2011", "AOD"],                     #
        ["HIRun2013", "AOD"],                     #
        ["HIRun2015", "AOD"],                     #
        ["PARun2016", "AOD"],                     #
        ["HIRun2018", "AOD"],                     #
        ["HINPbPbAutumn18DR", "AODSIM"],          # 
        ["HiWinter13", "AODSIM"],                 #
        ["pPb816Summer16DR", "AODSIM"],           #
        ["HINPbPbWinter16DR", "AODSIM"],          #
        ["HiFall13DR53X", "GEN-SIM-RECO"],        #    
        ["pAWinter13DR53X", "GEN-SIM-RECO"],      #
        ["XeXeRun2017", "*"],                     #
        ["HIRun2013A", "RECO"],                   #
        ["HINppWinter16DR", "AODSIM"],            #
        ["LowPU2010", "GEN-SIM"],                 #
        ["Run2010", "AOD"],                       # DPOA
        ["Run2010", "RAW-RECO"],                  # Keep cosmics
        ["Run2010", "RECO"],                      # Keep cosmics
        ["Run2011", "AOD"],                       #
        ["Run2011", "RAW-RECO"],                  #
        ["Run2011", "RECO"],                      #    
        ["Run2012", "AOD"],                       # DPOA
        ["Run2012", "RAW-RECO"],                  # Cosmics
        ["Summer11", "AODSIM"],                   #
        ["Summer11Leg", "GEN-SIM"],               #
        ["Fall11", "AODSIM"],                     #    
        ["Summer11LegDR", "AODSIM"],              #     
        ["Summer12DR53X", "AODSIM"],              #
        ["Summer11dr53X", "AODSIM"],              #
        ["Summer12_DR53X", "*"],                  #     
        ["Summer12", "AODSIM"],                   #
        ["Summer12", "GEN-SIM"],                  # 
        ["Summer13dr53X", "AODSIM"],              #
        ["Run2015C_25ns", "*"],                   #    
        ["Run2015", "AOD"],                       # DPOA
        ["Run2015", "RAW-RECO"],                  # Cosmics
        ["Phase2HLTTDRWinter20DIGI", "*"],        # Requested by TSG    
        ["Phase2HLTTDRSummer20ReRECOMiniAOD", "*"], #
        ["PhaseIITDRSpring17DR", "AOD"],          #
        ["PhaseIIFall16DR82", "AODSIM"],          #
        ["PhaseIISummer17wmLHEGENOnly", "LHE"],   # 
        ["GEM2019Upg14DR", "AODSIM"],             #
        ["PhaseISpring17DR", "AODSIM"],           # 
        ["PhaseIFall16DR", "AODSIM"],             #
        ["StreamExpressAlignment", "ALCARECO"],   #
        ["TkAlCosmicsInCollisions", "ALCARECO"],  #
        ["CMSSW_1_3_2", "RECO"],                  # What is this?
        ]

In [18]:
for ipair in campaign_tiers_to_keep :
    if ipair[1] not in global_exceptions[ipair[0]]:
        global_exceptions[ipair[0]].append(ipair[1])

In [19]:
datasets_strings = datasets.to_numpy(dtype="str")
tier_strings = tier

### Go through all data tiers in the global exception list and keep those requested. 

In [20]:
for icampaign,itiers in global_exceptions.items():
    if "*" in itiers: 
        keep = np.where((np.core.defchararray.find(datasets_strings, icampaign) == -1),keep, True)
    else: 
        for itier in itiers: 
            found_campaign_in_dataset = (np.core.defchararray.find(datasets_strings, icampaign) != -1)
            tier_should_be_kept = np.where(tier == itier, True, False)
            keep = np.where(found_campaign_in_dataset & tier_should_be_kept,
                True, 
                keep)

# Current size breakdown

In [21]:
print('Total size               : %12.1f' % np.sum(size_on_tape))
print('Total size to keep       : %12.1f' % np.sum(size_on_tape[keep]))
print('Total size to delete     : %12.1f' % np.sum(size_on_tape[keep == False]))

Total size               :      19915.4
Total size to keep       :       7432.0
Total size to delete     :      12483.4


In [22]:
datasets_kept = datasets[keep]
datasets_categories_kept = np.full_like(datasets_kept, "")
size_on_tape_kept = size_on_tape[keep]

In [23]:
np.sum(size_on_tape_kept)

7432.02764

## Invalid exceptions

In [24]:
exception_valid = np.in1d(exception_list, datasets[keep == True])
exception_names = exception_list[exception_valid == False]

### Exceptions that don't seem to be on tape.

These are going to be safe and off of the "to delete" list, but just in case people are curious. 

In [25]:
for i in exception_names: 
    if "*" not in i : 
        print(i)

/Single_Pion_gun_E_200to500_13TeV_pythia8/RunIIWinter19PFCalibDR-2016ConditionsNoPU_105X_mcRun2_asymptotic_newECALSR_v2-v2/GEN-SIM-RECO
/Single_Pion_gun_E_2to200_13TeV_pythia8/RunIIWinter19PFCalibDR-2016ConditionsNoPU_105X_mcRun2_asymptotic_newECALSR_v2-v2/GEN-SIM-RECO
/HighEGJet/Run2017H-17Nov2017-v1/AOD
/HighEGJet/Run2017G-17Nov2017-v2/AOD
/SingleMuon/Run2017G-17Nov2017-v1/AOD
/DYJetsToEE_M-50_massWgtFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/RunIILowPUSpring18DR-fixECALGT_94X_mc2017_realistic_v10For2017H_v2-v1/AODSIM
/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIILowPUSpring18DR-fixECALGT_94X_mc2017_realistic_v10For2017H_v2_ext1-v1/AODSIM
/DYJetsToMuMu_M-50_massWgtFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/RunIILowPUSpring18DR-fixECALGT_94X_mc2017_realistic_v10For2017H_v2-v1/AODSIM
/DYJetsToTauTau_M-50_massWgtFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/RunIILowPUSpring18DR-fixECALGT_94X_mc2017_realistic_v10For2017H_v2-v1/AODSIM
/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythi

# Make formatted lists to keep and drop

In [26]:
df_keep = df[keep == True]
df_drop = df[keep == False]

In [27]:
## Stupid delimeter. 
delim = ''.join( ['-' for i in range(75)])

# Keep list

In [28]:
print( "%30s : %12s | %12s | %12s" % ( "Category", "T0", "T1", "Total" ) )
print(delim)
for category in categories:
    isum_t0 = np.sum( df_keep[ (df_keep["category"] == category) & (df_keep["RSE Name"] == "T0_CH_CERN_Tape")]["dataset_size_in_rse_tb"] )
    isum_t1 = np.sum( df_keep[ (df_keep["category"] == category) & (df_keep["RSE Name"] != "T0_CH_CERN_Tape")]["dataset_size_in_rse_tb"] )
    print( "%30s : %12.1f | %12.1f | %12.1f" % ( category, isum_t0, isum_t1, isum_t0 + isum_t1 ) )
print(delim)
sum_t0 = np.sum( df_keep[(df_keep["RSE Name"] == "T0_CH_CERN_Tape")]["dataset_size_in_rse_tb"] )
sum_t1 = np.sum( df_keep[(df_keep["RSE Name"] != "T0_CH_CERN_Tape")]["dataset_size_in_rse_tb"] )
print( "%30s : %12.1f | %12.1f | %12.1f" % ( "Total", sum_t0, sum_t1, sum_t0 + sum_t1 ) )

                      Category :           T0 |           T1 |        Total
---------------------------------------------------------------------------
                         Skims :          0.0 |        118.6 |        118.6
       Commissioning + Cosmics :          0.0 |        526.4 |        526.4
                    Cosmics MC :          0.0 |          0.0 |          0.0
                         Run 3 :          0.0 |        463.4 |        463.4
               Run 2 legacy MC :          0.0 |        874.4 |        874.4
             Run 2 legacy data :          0.0 |       2626.4 |       2626.4
         Run 2 pre-legacy data :          0.0 |        259.9 |        259.9
           Run 2 pre-legacy MC :          0.0 |        762.6 |        762.6
                     B-Parking :          0.0 |          0.0 |          0.0
                           HIN :          0.0 |         92.6 |         92.6
                          ALCA :          0.0 |         43.2 |         43.2
            

In [29]:
keeping_sorted = df_keep.sort_values(by="dataset_size_in_rse_tb", ascending=False)

In [30]:
keeping_sorted.get("dataset", "size_on_tape_[TB]")

4       /QCD_Pt-15to7000_TuneCP5_Flat_14TeV-pythia8/Ph...
4288    /ggH_HToSSTobbbb_MH-125_TuneCP5_13TeV-powheg-p...
9       /ggH_HToSSTodddd_MH-125_TuneCP5_13TeV-powheg-p...
787                 /SingleMuon/Run2018A-17Sep2018-v2/AOD
11      /JetHT/Run2018D-JetHTJetPlusHOFilter-12Nov2019...
                              ...                        
3905    /BuToKstarPsi2S_EtaPtFilter_8TeV-pythia6-evtge...
4299    /BuToKstarPsi2S_EtaPtFilter_8TeV-pythia6-evtge...
4021    /BsToJPsiPhi_JPsiToMuMu_PhiToKK_EtaPtFilter_So...
3995    /TTToHadronic_hdampDOWN_TuneCP5_13TeV-powheg-p...
4144    /TTToSemiLeptonic_TuneCP5up_13TeV-powheg-pythi...
Name: dataset, Length: 1297, dtype: object

In [31]:
df_keep.to_csv("keeping_sorted_fall2022_disk.csv")

# Drop list

In [32]:
print( "%30s : %12s | %12s | %12s" % ( "Category", "T0", "T1", "Total" ) )
print(delim)
for category in categories:
    isum_t0 = np.sum( df_drop[ (df_drop["category"] == category) & (df_drop["RSE Name"] == "T0_CH_CERN_Tape")]["dataset_size_in_rse_tb"] )
    isum_t1 = np.sum( df_drop[ (df_drop["category"] == category) & (df_drop["RSE Name"] != "T0_CH_CERN_Tape")]["dataset_size_in_rse_tb"] )
    print( "%30s : %12.1f | %12.1f | %12.1f" % ( category, isum_t0, isum_t1, isum_t0 + isum_t1 ) )
print(delim)
sum_t0 = np.sum( df_drop[(df_drop["RSE Name"] == "T0_CH_CERN_Tape")]["dataset_size_in_rse_tb"] )
sum_t1 = np.sum( df_drop[(df_drop["RSE Name"] != "T0_CH_CERN_Tape")]["dataset_size_in_rse_tb"] )
print( "%30s : %12.1f | %12.1f | %12.1f" % ( "Total", sum_t0, sum_t1, sum_t0 + sum_t1 ) )    

                      Category :           T0 |           T1 |        Total
---------------------------------------------------------------------------
                         Skims :          0.0 |         31.8 |         31.8
       Commissioning + Cosmics :          0.0 |         31.5 |         31.5
                    Cosmics MC :          0.0 |          0.0 |          0.0
                         Run 3 :          0.0 |          0.0 |          0.0
               Run 2 legacy MC :          0.0 |        375.4 |        375.4
             Run 2 legacy data :          0.0 |          0.0 |          0.0
         Run 2 pre-legacy data :          0.0 |       1597.9 |       1597.9
           Run 2 pre-legacy MC :          0.0 |        853.8 |        853.8
                     B-Parking :          0.0 |        199.8 |        199.8
                           HIN :          0.0 |       1888.3 |       1888.3
                          ALCA :          0.0 |        162.0 |        162.0
            

In [33]:
dropping_sorted = df_drop.sort_values(by="dataset_size_in_rse_tb", ascending=False)
dropping_sorted_values = dropping_sorted["dataset"]

In [34]:
df_drop.to_csv("dropping_sorted_fall2022_disk.csv")

In [35]:
dropping_sorted

,dataset,RSE Name,dataset_size_in_rse_tb,last_access_time_of_dataset_in_rse,last_access_time_of_dataset_in_rse.1,#_files_of_dataset_in_rse,#_accessed_files_of_dataset_in_rse,unique_accounts,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,category
6,/HIExpressPhysics/HIRun2018A-Express-v1/FEVT,T2_CH_CERN,190.46522,1/1/70,0,42760,42760,['sync_t2_ch_cern'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIN
3112,/HIAllPhysics/HIRun2010-ZS-v2/RECO,T3_CH_CERN_OpenData,184.06570,1/1/70,0,31600,31600,['t3_ch_cern_opendata_local'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIN
54,/HIMinimumBias9/HIRun2018A-04Apr2019-v1/AOD,T2_US_Vanderbilt,103.46607,10/6/21,1633503356000,34727,34727,['transfer_ops'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIN
4009,/HIMinimumBias17/HIRun2018A-04Apr2019-v1/AOD,T2_US_Vanderbilt,102.55292,10/4/21,1633323868000,34719,34719,['transfer_ops'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIN
4236,/HIMinimumBias16/HIRun2018A-04Apr2019-v1/AOD,T2_US_Vanderbilt,102.44592,10/6/21,1633506873000,34718,34718,['transfer_ops'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4301,/SinglePhoton/Run2017C-17Nov2017-v1/AOD,T2_BE_UCL,1.00435,1/1/70,0,277,277,['sync_t2_be_ucl'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Run 2 pre-legacy data
821,/ScoutingMonitor/Run2018A-PromptReco-v1/MINIAOD,T2_US_UCSD,1.00222,1/1/70,0,422,422,['transfer_ops'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prompt
3136,/DYToLL_2J_13TeV-amcatnloFXFX-pythia8/RunIIFal...,T3_CH_CERN_OpenData,1.00173,1/1/70,0,947,947,['kati'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miscellaneous
3777,/DoubleJpsiToMuMu_DPS_13TeV-pythia8/RunIISumme...,T2_US_Wisconsin,1.00085,1/1/70,0,406,406,['transfer_ops'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miscellaneous


# Total to remove from T0

In [36]:
np.sum( dropping_sorted["dataset_size_in_rse_tb"][dropping_sorted["RSE Name"] == "T0_CH_CERN_Tape"] )

0.0

# Total to remove from T1

In [37]:
np.sum( dropping_sorted["dataset_size_in_rse_tb"] ) - np.sum( dropping_sorted["RSE Name"][dropping_sorted["RSE Name"] == "T0_CH_CERN_Tape"] )

12483.39966

## DPOA Data

In [38]:
vals_dpoa = keeping_sorted["dataset_size_in_rse_tb"][keeping_sorted["category"] == "DPOA"][keeping_sorted["RSE Name"] == "T0_CH_CERN_Tape"]

In [39]:
np.sum( vals_dpoa )

0.0

## Sanity check of exceptions

List of exceptions should not be in the list to drop. The intersection of these sets should be empty. 

In [40]:
set(exception_list) & set(dropping_sorted_values)

set()